In [11]:
import os
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [2]:
from google.colab import drive
import zipfile

drive.mount('/content/drive')

!unzip "/content/drive/MyDrive/dataset.zip" -d "/content/sample_data"

Mounted at /content/drive
Archive:  /content/drive/MyDrive/dataset.zip
   creating: /content/sample_data/dataset/
   creating: /content/sample_data/dataset/Test_Train_Splits/
   creating: /content/sample_data/dataset/Test_Train_Splits/50%/
  inflating: /content/sample_data/dataset/Test_Train_Splits/50%/fall_test_split1.txt  
  inflating: /content/sample_data/dataset/Test_Train_Splits/50%/fall_test_split2.txt  
  inflating: /content/sample_data/dataset/Test_Train_Splits/50%/fall_test_split3.txt  
  inflating: /content/sample_data/dataset/Test_Train_Splits/50%/fall_test_split4.txt  
  inflating: /content/sample_data/dataset/Test_Train_Splits/50%/fall_test_split5.txt  
  inflating: /content/sample_data/dataset/Test_Train_Splits/50%/grab_test_split1.txt  
  inflating: /content/sample_data/dataset/Test_Train_Splits/50%/grab_test_split2.txt  
  inflating: /content/sample_data/dataset/Test_Train_Splits/50%/grab_test_split3.txt  
  inflating: /content/sample_data/dataset/Test_Train_Splits/50%/

In [12]:
import os
import pandas as pd

# 🔥 Ana dataset klasörü
dataset_path = "/content/sample_data/dataset/Videos/Videos/"

# 📌 Normal ve Abnormal klasörlerini al
normal_path = os.path.normpath(os.path.join(dataset_path, "normal"))
abnormal_path = os.path.normpath(os.path.join(dataset_path, "abnormal"))

# 📌 Tüm abnormal sınıfları al
abnormal_classes = [folder for folder in os.listdir(abnormal_path) if os.path.isdir(os.path.join(abnormal_path, folder))]
class_to_index = {cls: idx for idx, cls in enumerate(abnormal_classes, start=1)}  # 1, 2, 3... (Abnormal sınıflar)

# 📌 DataFrame oluşturmak için liste
data = []

# ✅ Normal videolar (Binary = 0, Multi = -1)
for subfolder in os.listdir(normal_path):
    subfolder_path = os.path.normpath(os.path.join(normal_path, subfolder))

    if os.path.isdir(subfolder_path):  # Eğer bir klasörse
        videos = [
            os.path.normpath(os.path.join(subfolder_path, v)).replace("\\", "/")  # Windows uyumu için düzeltilmiş
            for v in os.listdir(subfolder_path) if v.endswith(('.mp4', '.avi', '.mov'))
        ]
        for video in videos:
            data.append((video, "normal", 0, -1))  # Binary label = 0, Multi label = -1 (yok)

# ✅ Anormal videolar (Binary = 1, Multi = class index)
for subfolder in abnormal_classes:
    subfolder_path = os.path.normpath(os.path.join(abnormal_path, subfolder))

    if os.path.isdir(subfolder_path):  # Eğer bir klasörse
        videos = [
            os.path.normpath(os.path.join(subfolder_path, v)).replace("\\", "/")  # Windows uyumu için düzeltilmiş
            for v in os.listdir(subfolder_path) if v.endswith(('.mp4', '.avi', '.mov'))
        ]
        for video in videos:
            data.append((video, subfolder, 1, class_to_index.get(subfolder, -1)))  # Binary label = 1, Multi label = class index

# 🔥 DataFrame oluştur
df = pd.DataFrame(data, columns=["video_path", "category", "binary_label", "multi_label"])


In [13]:
train_df, test_df = train_test_split(df, test_size=0.2,stratify=df["binary_label"],random_state=42)

print(f"Train set {len(train_df)} videos")
print(f"Test set {len(test_df)} videos")

Train set 2224 videos
Test set 556 videos


# Buraya preprocessing adımı olarak Veri döndürme, zoomout, kontrast vs eklenebilir

In [14]:
import torch
import torchvision
import torch.nn.functional as F
from torch.utils.data import Dataset

class VideoDataset(Dataset):
    def __init__(self, df, num_frames=8, transform=None, resize_size=(192, 192)):
        self.df = df
        self.num_frames = num_frames
        self.transform = transform
        self.resize_size = resize_size  # Sabit video boyutu (H, W)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        video_path = self.df.iloc[idx]["video_path"]
        binary_label = self.df.iloc[idx]["binary_label"]
        multi_label = self.df.iloc[idx]["multi_label"]

        # 🎥 Video'dan frame'leri oku
        frames = self.extract_frames(video_path, self.num_frames)

        #print(f"📌 İlk Frame Boyutu (T, C, H, W): {frames.shape}")

        # **🎯 GİRİŞ FORMATINI DÜZELTME**
        frames = frames.permute(1, 0, 2, 3).contiguous()  # (T, C, H, W) → (C, T, H, W)
        #print(f"✅ Permute Sonrası Boyut (C, T, H, W): {frames.shape}")

        # Eğer transform varsa uygula
        if self.transform:
            frames = self.transform(frames)

        return frames, torch.tensor(binary_label, dtype=torch.float32), torch.tensor(multi_label, dtype=torch.long)

    def extract_frames(self, video_path, num_frames):
        vr = torchvision.io.VideoReader(video_path, "video")
        vr.set_current_stream("video")

        frames = []
        for frame in vr:
            frame = frame['data'].float() / 255.0  # Normalize (C, H, W)
            #print(f"🔹 Orijinal Frame Boyutu: {frame.shape}")

            frame = F.interpolate(frame.unsqueeze(0), size=self.resize_size, mode="bilinear", align_corners=False)
            frame = frame.squeeze(0)  # (C, H, W)
            #print(f"🟡 Resize Sonrası Boyut: {frame.shape}")

            frames.append(frame)

            if len(frames) >= num_frames:
                break

        # Eğer video kısa ise son frame ile tamamla
        while len(frames) < num_frames:
            frames.append(frames[-1].clone())  # Tensor'ü kopyalayarak ekle

        frames = torch.stack(frames)  # Frame'leri birleştir
        frames = frames[:num_frames]  # Sabit sayıda frame al (T, C, H, W)
        #print(f"🔵 Stack Sonrası Boyut (T, C, H, W): {frames.shape}")

        return frames  # GPU'ya göndermeyi __getitem__ içinde yapacağız

In [15]:
train_dataset = VideoDataset(train_df, num_frames=32)
test_dataset = VideoDataset(test_df, num_frames=32)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

print(f"Train loader {len(train_loader)} batches")
print(f"Test loader {len(test_loader)} batches")

Train loader 139 batches
Test loader 35 batches


X3D modelimizin output layerini tüm classlarımızı kapsayacak şekilde arttırmamız lazım
Son katman ise ResNetBasicHead altında proj olarak tanımlanmış.

In [21]:
!pip install pytorchvideo torch torchvision pyav
!pip install --upgrade Pillow

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

# ✅ X3D Modelini Yükleme
model_name = "x3d_xs"
base_model = torch.hub.load('facebookresearch/pytorchvideo', model_name, pretrained=True)

# ✅ **Çıkış Katmanlarını Güncelle**
in_features = base_model.blocks[-1].proj.out_features  # ✅ **Doğru çıkışı al!** (400 olmalı)

binary_head = nn.Linear(2048, 1)  # Binary sınıflandırma için Sigmoid
multi_head = nn.Linear(2048, len(train_df["multi_label"].unique()))  # Multi-class sınıflandırma

# ✅ **MultiTaskX3D Modeli**
class MultiTaskX3D(nn.Module):
    def __init__(self, base_model, binary_head, multi_head):
        super(MultiTaskX3D, self).__init__()

        # **Feature Extractor - Son bloğun içindeki ProjectedPool eklendi!**
        self.feature_extractor = nn.Sequential(
            *base_model.blocks[:-1],
            base_model.blocks[-1].pool  # ✅ **Burada 400'e dönüşüm sağlandı!**
        )

        # **Global Average Pooling - Tüm boyutları 1x1x1'e düşürüyor!**
        self.gap = nn.AdaptiveAvgPool3d(1)

        # **Flatten**
        self.flatten = nn.Flatten()

        # **Çıkış Katmanları**
        self.binary_head = binary_head
        self.binary_activation = nn.Sigmoid()

        self.multi_head = multi_head
        self.multi_activation = nn.Softmax(dim=1)

    def forward(self, x):
        #print(f"🚀 Model Girişi: {x.shape}")

        x = self.feature_extractor(x)
        #print(f"🟡 Feature Extractor Çıkışı: {x.shape}")  # **(16, 400, 1, 1, 1) olmalı!**

        x = self.gap(x)  # ✅ **GAP ile (16, 400, 1, 1, 1)**
        #print(f"🔵 GAP Sonrası Boyut: {x.shape}")

        x = self.flatten(x)  # ✅ **Artık (16, 400) olması lazım!**
        #print(f"🟢 Flatten Sonrası Boyut: {x.shape}")

        binary_out = self.binary_head(x)  # Binary sınıflandırma
        multi_out = self.multi_activation(self.multi_head(x))  # Multi-class sınıflandırma

        return binary_out, multi_out

# ✅ Modeli başlat
model = MultiTaskX3D(base_model, binary_head, multi_head)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

print(f"🚀 Model başarıyla yüklendi! Çalıştırılan cihaz: {device}")

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main


🚀 Model başarıyla yüklendi! Çalıştırılan cihaz: cuda


import torch

model = model.to("cpu")
dummy_input = torch.randn(1, 3, 16, 160, 160)

torch.onnx.export(model, dummy_input, "x3d_xs.onnx", opset_version=11
                  ,do_constant_folding=True, input_names=["input"], output_names=["output"])

In [17]:
from torch.nn import CrossEntropyLoss

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self,inputs,targets):
        ce_loss = F.cross_entropy(inputs,targets,reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1-pt)**self.gamma * ce_loss
        return focal_loss.mean()

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision

# 🎯 Loss & Optimizer
criterion_binary = nn.BCEWithLogitsLoss()  # Binary classification için
criterion_multi = FocalLoss()  # Multi-class classification için

optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-2)  # Weight decay ile overfitting azaltılır
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

# ⚡ AMP (Automatic Mixed Precision)
scaler = torch.amp.GradScaler()

# 🛑 Early Stopping Parametreleri
patience = 3  # Kaç epoch boyunca iyileşme olmazsa durdursun
min_delta = 0.01  # İyileşme için gereken minimum fark
best_loss = float("inf")  # En iyi validation loss başta sonsuz olarak ayarlanır
counter = 0  # İyileşme olmayan epoch sayısı
accuracy_threshold = 85.0  # Accuracy'nin geçmesi gereken eşik

device = "cuda" if torch.cuda.is_available() else "cpu"

for epoch in range(40):
    model.train()
    running_loss = 0.0

    for videos, binary_labels, multi_labels in train_loader:
        videos, binary_labels, multi_labels = (
            videos.to(device),
            binary_labels.to(device).float(),
            multi_labels.to(device),
        )

        optimizer.zero_grad()

        with torch.amp.autocast(device):
            binary_out, multi_out = model(videos)

            # 🟢 Binary Classification Loss (Sigmoid)
            binary_loss = criterion_binary(binary_out.squeeze(), binary_labels)

            # 🔴 Multi-Class Classification Loss (Softmax) → Sadece anormal olanlara uygula
            mask = (binary_labels == 1)
            if mask.sum() > 0:
                multi_loss = criterion_multi(multi_out[mask], multi_labels[mask])
            else:
                multi_loss = torch.tensor(0.0, device=device)

            # ✅ Toplam loss hesapla (Binary + Multi)
            total_loss = binary_loss + multi_loss

        scaler.scale(total_loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += total_loss.item()

    train_loss = running_loss / len(train_loader)

    # 📌 VALIDATION AŞAMASI
    model.eval()
    val_loss = 0.0
    correct_binary = 0
    correct_multi = 0
    total_binary = 0
    total_multi = 0

    with torch.no_grad():
        for videos, binary_labels, multi_labels in test_loader:
            videos, binary_labels, multi_labels = (
                videos.to(device),
                binary_labels.to(device).float(),
                multi_labels.to(device),
            )

            binary_out, multi_out = model(videos)

            # ✅ Binary Accuracy (Normal mi, Anormal mi?)
            pred_binary = (binary_out.squeeze() > 0.5).long()
            correct_binary += (pred_binary == binary_labels).sum().item()
            total_binary += binary_labels.size(0)

            # ✅ Multi-Class Accuracy (Eğer anormalse hangi sınıf?)
            mask = (binary_labels == 1)
            if mask.sum() > 0:
                pred_multi = torch.argmax(multi_out[mask], dim=1)
                correct_multi += (pred_multi == multi_labels[mask]).sum().item()
                total_multi += multi_labels[mask].size(0)

            # ✅ Validation loss hesapla
            loss = criterion_binary(binary_out.squeeze(), binary_labels)
            if total_multi > 0:
                loss += criterion_multi(multi_out[mask], multi_labels[mask])
            val_loss += loss.item()

    val_loss /= len(test_loader)
    binary_acc = 100 * correct_binary / total_binary
    multi_acc = 100 * correct_multi / total_multi if total_multi > 0 else 0

    print(
        f"Epoch {epoch+1} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} "
        f"| Binary Acc: {binary_acc:.2f}% | Multi Acc: {multi_acc:.2f}%"
    )

    # ✅ **Early Stopping (Binary Accuracy 85%'i geçmeli)**
    if binary_acc >= accuracy_threshold:
        if best_loss - val_loss > min_delta:
            best_loss = val_loss
            counter = 0
            torch.save(model.state_dict(), f"best_model_epoch_{epoch+1}.pth")
            print(f"✅ Model kaydedildi: best_model_epoch_{epoch+1}.pth")
        else:
            counter += 1
            print(f"❗ Early stopping counter: {counter}/{patience}")

        if counter >= patience:
            print("⏹️ Early stopping triggered. Training stopped.")
            break
    else:
        print(f"🚨 Accuracy {binary_acc:.2f}% < {accuracy_threshold}%, early stopping devre dışı!")

    scheduler.step()

    if (epoch + 1) % 5 == 0:
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
            },
            f"checkpoint_{epoch}.pth",
        )
        print(f"Model {epoch + 1} saved to checkpoint_{epoch + 1}.pth")

print("✅ Eğitim tamamlandı!")

AttributeError: 'ImportError' object has no attribute 'open'